In [20]:
import requests 
from collections import defaultdict

In [15]:
docs_url = 'https://raw.githubusercontent.com/AdairPonceuwu/ch_llm/main/documents.json'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [16]:
documents[0]

{'text': 'El Eje 4 contribuirá al repoblamiento y mejoramiento urbano del Centro Histórico de Puebla, protegiendo su patrimonio histórico cultural.',
 'section': 'Vinculación a otros instrumentos municipales',
 'question': '¿Cómo contribuirá el Eje 4 a la protección del patrimonio histórico cultural del Centro Histórico?',
 'topic': 'PMD'}

In [17]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['topic']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [18]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [19]:
documents[3]

{'text': 'En el último decenio, la zona metropolitana registró su menor crecimiento poblacional con una tasa media anual de 0.9%. Esta situación contrasta con las altas tasas de crecimiento de municipios como Cuautlancingo, San Andrés Cholula, Huejotzingo y Coronango.',
 'section': 'Densidad poblacional',
 'question': '¿Cuál fue la tasa media anual de crecimiento poblacional en la zona metropolitana en el último decenio?',
 'topic': 'PMD',
 'id': 'bbcc69fc'}

In [21]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [22]:
len(hashes), len(documents)

(186, 187)

In [23]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

c78f3aa6 2


In [24]:
hashes['c78f3aa6']

[{'text': 'C6 Reservas ecológica, parques, jardines, espacio públcio y área verde',
  'section': 'Intervención Arquitectónica ',
  'question': '¿Qué establece la categoría 6 en el Programa Parcial de Desarrollo Urbano Sustentable?',
  'topic': 'PPDUS',
  'id': 'c78f3aa6'},
 {'text': 'C6 Reservas ecológica, parques, jardines, espacio públcio y área verde',
  'section': 'Intervención Arquitectónica ',
  'question': '¿Qué establece la categoría 6 en el Programa Parcial de Desarrollo Urbano Sustentable?',
  'topic': 'PPDUS',
  'id': 'c78f3aa6'}]

In [25]:
import json

In [29]:
with open('documents-with-ids.json', 'w', encoding='utf-8') as f_out:
    json.dump(documents, f_out, ensure_ascii=False, indent=2)

In [30]:
prompt_template = """
Eres una persona que necesita informacion sobre el Centro Histórico.
Formula 5 preguntas que esta persona puede preguntar con base a un registro FAQ. El registro 
debe contener la respuesta a la preguntas, y las preguntas deben de ser completas y no tan cortas.
Si es posible, usa la menor cantidad posible de palabras del registro. 

El registro:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [31]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [32]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [35]:
from tqdm.auto import tqdm

In [36]:
results = {}

In [37]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|                                                                   | 0/187 [00:02<?, ?it/s]


APIConnectionError: Connection error.

In [38]:
import pickle

In [ ]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)